# Ensemble Learning

## Initial Imports

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [30]:
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [31]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df_LoanStats = pd.read_csv((file_path), index_col = "issue_d").dropna()



In [32]:
df_LoanStats["home_ownership"].value_counts()

MORTGAGE    36219
RENT        24613
OWN          7346
ANY           639
Name: home_ownership, dtype: int64

In [33]:
df_LoanStats["home_ownership"] = df_LoanStats["home_ownership"].fillna("ANY")

In [34]:
# Preview the data
df_LoanStats.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
issue_d,,,,,,,,,,,,,,,,,,,,,
Mar-2019,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
Mar-2019,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
Mar-2019,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
Mar-2019,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
Mar-2019,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [35]:
# Create our features
X = pd.get_dummies(df_LoanStats.drop("loan_status", axis=1))

# Create our target
y = df_LoanStats[["loan_status"]]

In [36]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.148001,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.355104,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [37]:
# Check the balance of our target values
y["loan_status"].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [38]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, = train_test_split(X, y["loan_status"], 
    random_state=1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [39]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler

StandardScaler()

In [40]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
X_scaler

StandardScaler()

In [41]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape

(51612, 92)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [42]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
bal_ran_class = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
bal_ran_class.fit(X_train, y_train)


BalancedRandomForestClassifier(random_state=1)

In [43]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = bal_ran_class.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7248057096583149

In [51]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   52,    35],
       [ 2535, 14583]], dtype=int64)

In [52]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.60      0.85      0.04      0.71      0.50        87
   low_risk       1.00      0.85      0.60      0.92      0.71      0.52     17118

avg / total       0.99      0.85      0.60      0.91      0.71      0.52     17205



In [54]:
# List the features sorted in descending order by feature importance
feat_importance = bal_ran_class.feature_importances_ 
sorted(zip(bal_ran_class.feature_importances_, X.columns), reverse=True)

[(0.06396532338163047, 'total_rec_int'),
 (0.06198436391890879, 'last_pymnt_amnt'),
 (0.06131163845690018, 'total_pymnt'),
 (0.06042218847896876, 'total_rec_prncp'),
 (0.056544921853939153, 'total_pymnt_inv'),
 (0.03068970474987624, 'int_rate'),
 (0.019975607441453276, 'dti'),
 (0.019021850689876284, 'revol_bal'),
 (0.018053492360663247, 'il_util'),
 (0.017067149069837823, 'installment'),
 (0.0169133332307869, 'mo_sin_old_rev_tl_op'),
 (0.016682895449532355, 'annual_inc'),
 (0.01642482463261731, 'mths_since_recent_inq'),
 (0.016378507530990547, 'mo_sin_old_il_acct'),
 (0.01617875285596531, 'out_prncp_inv'),
 (0.01615216788226461, 'bc_util'),
 (0.01515125751828634, 'tot_hi_cred_lim'),
 (0.015021455036263487, 'tot_cur_bal'),
 (0.014570180968198647, 'loan_amnt'),
 (0.014550708888984043, 'mths_since_rcnt_il'),
 (0.014545203730956021, 'max_bal_bc'),
 (0.014354986403767029, 'bc_open_to_buy'),
 (0.013652250904277423, 'total_bal_ex_mort'),
 (0.01361629027971775, 'total_il_high_credit_limit'),


### Easy Ensemble Classifier

In [56]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
easy_ensem_class = EasyEnsembleClassifier(n_estimators=100, random_state=1)
easy_ensem_class.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [57]:
# Calculated the balanced accuracy score
y_pred = easy_ensem_class.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7633354283250944

In [58]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   64,    23],
       [ 3577, 13541]], dtype=int64)

In [59]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.74      0.79      0.03      0.76      0.58        87
   low_risk       1.00      0.79      0.74      0.88      0.76      0.59     17118

avg / total       0.99      0.79      0.74      0.88      0.76      0.59     17205



### Final Questions

1. Which model had the best balanced accuracy score?

The Easy Ensemble Classifier

2. Which model had the best recall score?

The Balanced Random Forest Classifier

3. Which model had the best geometric mean score?

The Easy Ensemble Classifier

4. What are the top three features?

If you mean the types listed in the Imbalanced Classification Report, the three highest scoring features were precision, the f1 score, and the recall, in that order. However, if you look at the f1 score, it is defined as the numerical average between precision and recall. So, should the f1 score be included in this? 